<a href="https://colab.research.google.com/github/timider/dscproj/blob/main/dscproj-small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import seaborn as sns
from scipy import optimize
from scipy import stats
from sklearn.preprocessing import StandardScaler
import pymc3 as pm
import statsmodels.api as sm # check the error that cannot import name 'factorial' in from scipy.misc import factorial
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import tensorflow as tf
import altair as alt
# from linearmodels.iv import IV2SLS
from tensorflow.keras import datasets, layers, models
from tensorflow import keras
from numpy import mean
from numpy import std
from numpy import absolute
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn import datasets
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
cpi = pd.read_csv("https://raw.githubusercontent.com/timider/dscproj/main/CPI.csv")
ppico = pd.read_csv("https://raw.githubusercontent.com/timider/dscproj/main/PPICO.csv")
ppimanu = pd.read_csv("https://raw.githubusercontent.com/timider/dscproj/main/PPIMANU.csv")
gdp = pd.read_csv("https://raw.githubusercontent.com/timider/dscproj/main/GDPCR.csv")
unemp = pd.read_csv("https://raw.githubusercontent.com/timider/dscproj/main/UNRATE.csv")
govspend = pd.read_csv("https://raw.githubusercontent.com/timider/dscproj/main/FGEXPND.csv")
savings = pd.read_csv("https://raw.githubusercontent.com/timider/dscproj/main/PSAVERT.csv")
prod = pd.read_csv("https://raw.githubusercontent.com/timider/dscproj/main/PRS85006091.csv")
aaa = pd.read_csv("https://raw.githubusercontent.com/timider/dscproj/main/AAA.csv")
bbb = pd.read_csv("https://raw.githubusercontent.com/timider/dscproj/main/BBB.csv")
effr = pd.read_csv("https://raw.githubusercontent.com/timider/dscproj/main/EFFR.csv")
ts10 = pd.read_csv("https://raw.githubusercontent.com/timider/dscproj/main/TS10.csv")

In [6]:
df = pd.merge(cpi, ppico, how='inner', on='DATE')
df = pd.merge(df, ppimanu, how='inner', on='DATE')
df = pd.merge(df, gdp, how='inner', on='DATE')
df = pd.merge(df, unemp, how='inner', on='DATE')
df = pd.merge(df, govspend, how='inner', on='DATE')
df = pd.merge(df, aaa, how='inner', on='DATE')
df = pd.merge(df, bbb, how='inner', on='DATE')
df = pd.merge(df, effr, how='inner', on='DATE')
df = pd.merge(df, ts10, how='inner', on='DATE')
df = pd.merge(df, savings, how='inner', on='DATE')
df = pd.merge(df, prod, how='inner', on='DATE')

In [7]:
df = df.rename({
    'PPIACO_PC1':'PPICO',
    'CPIAUCSL_PC1':'CPI',
    'PCUOMFGOMFG_PC1':'PPIMANU',
    'GDPC1_PC1':'GDP',
    'UNRATE':'U',
    'FGEXPND_PC1':'GOVSPEND',
    'BAMLC0A4CBBBEY':'BBB',
    'DGS10':'BOND10',
    'PSAVERT':'S',
    'PRS85006091_NBD20000701':'PROD'
}, axis='columns')

In [8]:
df

,DATE,CPI,PPICO,PPIMANU,GDP,U,GOVSPEND,AAA,BBB,EFFR,BOND10,S,PROD
0,2000-07-01,3.45650,5.23438,3.85505,3.97408,3.9,4.09135,7.62,8.13,6.60,5.80,4.5,100.00000
1,2000-10-01,3.43602,6.57277,2.91188,2.90162,3.9,3.39396,7.21,7.97,5.41,5.12,4.4,83.33333
2,2001-01-01,2.98246,3.89908,1.20391,2.19554,4.3,6.23252,6.98,7.35,5.29,4.93,5.3,80.00000
3,2001-04-01,3.19396,1.27055,1.19225,0.99193,4.5,4.97400,7.18,7.35,3.95,5.42,4.4,73.33333
4,2001-07-01,2.59217,-1.03935,0.66815,0.48735,5.0,6.20687,7.17,6.85,2.75,4.60,7.0,93.33333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,2021-01-01,2.65838,11.34127,8.11232,0.54698,6.0,64.41446,3.04,2.55,0.06,1.74,26.6,130.00000
83,2021-04-01,5.34102,19.71757,14.56362,12.22668,5.9,-17.74758,2.79,2.28,0.08,1.45,9.5,73.33333
84,2021-07-01,5.38991,20.55141,15.01763,4.94656,4.7,-8.96970,2.53,2.38,0.06,1.52,8.1,-13.33333
85,2021-10-01,7.09654,20.36808,15.13472,5.53348,3.9,0.13917,2.65,2.60,0.07,1.52,8.7,63.33333


In [9]:
cpipred = []
for i in range(1, len(df['CPI'])):
    cpipred.append(df['CPI'][i])
cpipred = pd.DataFrame (cpipred, columns = ['CPIPRED'])

In [10]:
datepred = []
for i in range(1, len(df['DATE'])):
    datepred.append(df['DATE'][i])
datepred = pd.DataFrame (datepred, columns = ['DATEPRED'])

In [11]:
df = df.iloc[:-1 , :]

In [12]:
df = pd.concat([df, datepred], axis=1) 
df = df. T. drop_duplicates(). T

In [13]:
df = pd.concat([df, cpipred], axis=1) 
df = df. T. drop_duplicates(). T
df

,DATE,CPI,PPICO,PPIMANU,GDP,U,GOVSPEND,AAA,BBB,EFFR,BOND10,S,PROD,DATEPRED,CPIPRED
0,2000-07-01,3.45650,5.23438,3.85505,3.97408,3.9,4.09135,7.62,8.13,6.60,5.8,4.5,100.0,2000-10-01,3.43602
1,2000-10-01,3.43602,6.57277,2.91188,2.90162,3.9,3.39396,7.21,7.97,5.41,5.12,4.4,83.33333,2001-01-01,2.98246
2,2001-01-01,2.98246,3.89908,1.20391,2.19554,4.3,6.23252,6.98,7.35,5.29,4.93,5.3,80.0,2001-04-01,3.19396
3,2001-04-01,3.19396,1.27055,1.19225,0.99193,4.5,4.974,7.18,7.35,3.95,5.42,4.4,73.33333,2001-07-01,2.59217
4,2001-07-01,2.59217,-1.03935,0.66815,0.48735,5.0,6.20687,7.17,6.85,2.75,4.6,7.0,93.33333,2001-10-01,1.60367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,2020-10-01,1.27815,0.75377,0.15275,-2.26292,6.7,23.45544,2.26,2.06,0.09,0.93,14.0,90.0,2021-01-01,2.65838
82,2021-01-01,2.65838,11.34127,8.11232,0.54698,6.0,64.41446,3.04,2.55,0.06,1.74,26.6,130.0,2021-04-01,5.34102
83,2021-04-01,5.34102,19.71757,14.56362,12.22668,5.9,-17.74758,2.79,2.28,0.08,1.45,9.5,73.33333,2021-07-01,5.38991
84,2021-07-01,5.38991,20.55141,15.01763,4.94656,4.7,-8.9697,2.53,2.38,0.06,1.52,8.1,-13.33333,2021-10-01,7.09654


dataset is prepared, now lasso

for lasso and ridge we need to standardize variables

In [14]:
df.describe()

,DATE,CPI,PPICO,PPIMANU,GDP,U,GOVSPEND,AAA,BBB,EFFR,BOND10,S,PROD,DATEPRED,CPIPRED
count,86,86,86.00000,86.00000,86.00000,86.0,86.00000,86.00,86.0,86,86.00,86.0,86.0,86,86
unique,86,86,86.00000,86.00000,86.00000,42.0,86.00000,80.00,80.0,59,76.00,47.0,42.0,86,86
top,2000-07-01,3.45650,5.23438,3.85505,3.97408,5.0,4.09135,5.13,3.3,0.09,1.52,6.5,70.0,2000-10-01,3.43602
freq,1,1,1.00000,1.00000,1.00000,8.0,1.00000,3.00,2.0,10,2.00,7.0,5.0,1,1


In [15]:
df_x = df[['AAA', 'BBB', 'BOND10', 'GDP', 'GOVSPEND', 'PPICO', 'PPIMANU', 'PROD', 'S', 'U']]
dfst = df[['AAA', 'BBB', 'BOND10', 'GDP', 'GOVSPEND', 'PPICO', 'PPIMANU', 'PROD', 'S', 'U']] = (df_x-df_x.mean())/df_x.std()
dfst

,AAA,BBB,BOND10,GDP,GOVSPEND,PPICO,PPIMANU,PROD,S,U
0,2.34202,1.977854,2.108279,0.856702,-0.214227,0.361408,0.296891,0.757087,-0.660183,-1.110751
1,2.013162,1.879369,1.5757,0.404236,-0.267199,0.567608,0.093301,0.414405,-0.689555,-1.110751
2,1.82868,1.497741,1.426891,0.106344,-0.051586,0.155685,-0.275375,0.345869,-0.425208,-0.895242
3,1.989099,1.497741,1.810661,-0.401454,-0.147182,-0.249281,-0.277892,0.208796,-0.689555,-0.787487
4,1.981078,1.189976,1.168433,-0.614334,-0.053535,-0.605157,-0.391022,0.620014,0.074113,-0.5181
...,...,...,...,...,...,...,...,...,...,...
81,-1.957202,-1.75841,-1.70593,-1.774661,1.256642,-0.328899,-0.502275,0.551478,2.13014,0.397816
82,-1.331569,-1.456801,-1.071534,-0.589177,4.367829,1.302269,1.21585,1.373913,5.830989,0.020674
83,-1.532092,-1.622994,-1.298664,4.338437,-1.87308,2.592768,2.608404,0.208796,0.808408,-0.033204
84,-1.740636,-1.561441,-1.243839,1.266987,-1.206325,2.721234,2.706405,-1.573146,0.397203,-0.679732


In [16]:
scale= StandardScaler()

In [17]:
effrst = scale.fit_transform(df['EFFR'].array.reshape(-1,1))
cpist = scale.fit_transform(df['CPI'].array.reshape(-1,1)) 
cpipredst = scale.fit_transform(df['CPIPRED'].array.reshape(-1,1))


In [18]:
dfst['EFFR']=effrst
dfst['CPI']=cpist
dfst['CPIPRED']=cpipredst
dfst['DATE']=df['DATE']
dfst['DATEPRED']=df['DATEPRED']

In [19]:
dfst

,AAA,BBB,BOND10,GDP,GOVSPEND,PPICO,PPIMANU,PROD,S,U,EFFR,CPI,CPIPRED,DATE,DATEPRED
0,2.34202,1.977854,2.108279,0.856702,-0.214227,0.361408,0.296891,0.757087,-0.660183,-1.110751,2.921827,0.874579,0.738393,2000-07-01,2000-10-01
1,2.013162,1.879369,1.5757,0.404236,-0.267199,0.567608,0.093301,0.414405,-0.689555,-1.110751,2.240301,0.859968,0.446195,2000-10-01,2001-01-01
2,1.82868,1.497741,1.426891,0.106344,-0.051586,0.155685,-0.275375,0.345869,-0.425208,-0.895242,2.171575,0.536405,0.582450,2001-01-01,2001-04-01
3,1.989099,1.497741,1.810661,-0.401454,-0.147182,-0.249281,-0.277892,0.208796,-0.689555,-0.787487,1.404142,0.687286,0.194758,2001-04-01,2001-07-01
4,1.981078,1.189976,1.168433,-0.614334,-0.053535,-0.605157,-0.391022,0.620014,0.074113,-0.5181,0.716888,0.257978,-0.442066,2001-07-01,2001-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,-1.957202,-1.75841,-1.70593,-1.774661,1.256642,-0.328899,-0.502275,0.551478,2.13014,0.397816,-0.806524,-0.679425,0.237412,2020-10-01,2021-01-01
82,-1.331569,-1.456801,-1.071534,-0.589177,4.367829,1.302269,1.21585,1.373913,5.830989,0.020674,-0.823706,0.305211,1.965655,2021-01-01,2021-04-01
83,-1.532092,-1.622994,-1.298664,4.338437,-1.87308,2.592768,2.608404,0.208796,0.808408,-0.033204,-0.812251,2.218968,1.997151,2021-04-01,2021-07-01
84,-1.740636,-1.561441,-1.243839,1.266987,-1.206325,2.721234,2.706405,-1.573146,0.397203,-0.679732,-0.823706,2.253846,3.096617,2021-07-01,2021-10-01


df is standardized, now lasso

In [20]:
X = dfst.iloc[:,:12]
X

,AAA,BBB,BOND10,GDP,GOVSPEND,PPICO,PPIMANU,PROD,S,U,EFFR,CPI
0,2.34202,1.977854,2.108279,0.856702,-0.214227,0.361408,0.296891,0.757087,-0.660183,-1.110751,2.921827,0.874579
1,2.013162,1.879369,1.5757,0.404236,-0.267199,0.567608,0.093301,0.414405,-0.689555,-1.110751,2.240301,0.859968
2,1.82868,1.497741,1.426891,0.106344,-0.051586,0.155685,-0.275375,0.345869,-0.425208,-0.895242,2.171575,0.536405
3,1.989099,1.497741,1.810661,-0.401454,-0.147182,-0.249281,-0.277892,0.208796,-0.689555,-0.787487,1.404142,0.687286
4,1.981078,1.189976,1.168433,-0.614334,-0.053535,-0.605157,-0.391022,0.620014,0.074113,-0.5181,0.716888,0.257978
...,...,...,...,...,...,...,...,...,...,...,...,...
81,-1.957202,-1.75841,-1.70593,-1.774661,1.256642,-0.328899,-0.502275,0.551478,2.13014,0.397816,-0.806524,-0.679425
82,-1.331569,-1.456801,-1.071534,-0.589177,4.367829,1.302269,1.21585,1.373913,5.830989,0.020674,-0.823706,0.305211
83,-1.532092,-1.622994,-1.298664,4.338437,-1.87308,2.592768,2.608404,0.208796,0.808408,-0.033204,-0.812251,2.218968
84,-1.740636,-1.561441,-1.243839,1.266987,-1.206325,2.721234,2.706405,-1.573146,0.397203,-0.679732,-0.823706,2.253846


In [21]:
y = dfst.iloc[:,12]
y


0     0.738393
1     0.446195
2     0.582450
3     0.194758
4    -0.442066
        ...   
81    0.237412
82    1.965655
83    1.997151
84    3.096617
85    4.037872
Name: CPIPRED, Length: 86, dtype: float64

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=44)

In [88]:
lasso = Lasso(alpha=0.5)

In [89]:
lasso.fit(X_train, y_train)

Lasso(alpha=0.5)

In [90]:
lasso.score(X_test, y_test)
lasso.score(X_train, y_train)

0.22308351320846764

In [91]:
lasso.coef_

array([ 0.        , -0.        ,  0.        ,  0.        , -0.        ,
        0.14759083,  0.        ,  0.        , -0.        , -0.        ,
        0.        ,  0.        ])